# Bibliotecas

In [ ]:
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, precision_score, recall_score, f1_score
from pylab import text

In [ ]:
data_folder = r"C:\Users\anaso\Desktop\workspace\doutorado\data"
saving_path = r"C:\Users\anaso\Desktop\workspace\doutorado\imagens"

# Gráficos

### Resultados online do modelo NB136

In [ ]:
data_teste = pd.read_csv(data_folder + "\\test_data.csv", sep=';')
data_teste.rename(columns={'Time':'time'}, inplace=True)
data_teste['time'] = pd.to_datetime(data_teste['time'])
data_teste.head()

In [ ]:
data_teste.columns

In [ ]:
# Normaliza os dados para manter a confidencialidade

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

cols = ['PDIT-301', 'PIT-302', 'PIT-303', 'PIT-304', 'PIT-305', 'PIT-306', 'VT-301', 'VT-302', 'VT-303', \
        'VT-304', 'VT-321', 'VT-322', 'VT-323', 'VT-324', 'ZT-301', 'ZT-302', 'TI-305', 'TI-306', 'TI-307', \
       'TI-308', 'TI-309', 'TI-310', 'Corrente', 'Potencia', 'FIT-323', 'time', 'rotulos_multi', 'rotulos_bin']

ct = ColumnTransformer([('somename', StandardScaler(), data_teste.columns[1:-2])], remainder='passthrough')
data_teste = pd.DataFrame(ct.fit_transform(data_teste), columns=cols)
data_teste.head()

In [ ]:
results_online_nb136_17_18 = pd.read_csv(data_folder + "\\results_online_nb136_17_18.csv", sep=';')
results_online_nb136_17_18 = results_online_nb136_17_18.dropna().drop(['Unnamed: 0'], axis=1)
results_online_nb136_17_18['time'] = pd.to_datetime(results_online_nb136_17_18['time'])

results_online_nb136_20_24 = pd.read_csv(data_folder + "\\results_online_nb136_20_24.csv", sep=';')
results_online_nb136_20_24 = results_online_nb136_20_24.dropna().drop(['Unnamed: 0'], axis=1)
results_online_nb136_20_24['time'] = pd.to_datetime(results_online_nb136_20_24['time'])

In [ ]:
results_online_nb136_17_18.head()

In [ ]:
results_online_nb136_20_24.head()

### Falha 17

In [ ]:
fault = 17

In [ ]:
print(results_online_nb136_17_18[results_online_nb136_17_18['real'] == fault].time.max())
print(data_teste[data_teste['rotulos_multi'] == fault].time.max())
max_date = data_teste[data_teste['rotulos_multi'] == fault].time.max()

In [ ]:
fault_predict = results_online_nb136_17_18[results_online_nb136_17_18['time'] <= max_date]
fault_vars = data_teste[data_teste['time'] <= max_date][24:]

print(fault_predict.shape)
print(fault_vars.shape)

In [ ]:
final_dataset = fault_predict.merge(fault_vars, how='left', on=['time'])
final_dataset.shape

In [ ]:
final_dataset.isnull().sum().sum()

In [ ]:
final_dataset_filter = final_dataset[19000:].copy()
final_dataset_filter['prediction'].iloc[:2070] = 0.0

In [ ]:
final_dataset_filter['prediction'] = final_dataset_filter['prediction'].astype('int')

In [ ]:
fig, ax1 = plt.subplots(figsize=(9,2), dpi=300)

ax2 = ax1.twinx()
plot1 = ax1.plot(final_dataset_filter['time'], final_dataset_filter['prediction'], color='red', label='PREDIÇÃO (M15)', linewidth=1)
plot2 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==fault, \
                                                 final_dataset_filter['VT-322'], None), color="black", label='FAULT 16')
plot3 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==0, \
                                                 final_dataset_filter['VT-322'], None), color="blue", label='NORMAL')

# ax1.set_xlabel('Timestamp')
ax2.set_ylabel('VT-322')
ax1.set_ylabel('Prediction (M5)')
ax2.set_ylim([-2, 2])
ax1.yaxis.set_ticks(np.arange(0, 2, 1))

# Esquema para unificar na mesma caixa as legendas de ambos os eixos
lns = plot1+plot2+plot3
labs = [l.get_label() for l in lns]
ax2.legend(loc='lower right', prop={'size': 8})
# ax1.legend(lns, labs, loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=True, ncol=5)

fig.tight_layout()
# fig.savefig(saving_path + '/ingles_nb136_online_falha17.png', format='png', bbox_inches='tight');

### Falha 18

In [ ]:
fault = 18

In [ ]:
print(results_online_nb136_17_18[results_online_nb136_17_18['real'] == fault].time.max())
print(data_teste[data_teste['rotulos_multi'] == fault].time.max())
max_date = data_teste[data_teste['rotulos_multi'] == fault].time.max()

In [ ]:
fault_predict = results_online_nb136_17_18[results_online_nb136_17_18['time'] <= max_date]
fault_vars = data_teste[data_teste['time'] <= max_date][24:]

print(fault_predict.shape)
print(fault_vars.shape)

In [ ]:
final_dataset = fault_predict.merge(fault_vars, how='left', on=['time'])
final_dataset.shape

In [ ]:
final_dataset.isnull().sum().sum()

In [ ]:
final_dataset_filter = final_dataset[30000:-1000].copy()

In [ ]:
fig, ax1 = plt.subplots(figsize=(9,2), dpi=300)

ax2 = ax1.twinx()
plot1 = ax1.plot(final_dataset_filter['time'], final_dataset_filter['prediction'].astype('int'), color='red', label='PREDIÇÃO (M15)', linewidth=1)
plot2 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==fault, \
                                                 final_dataset_filter['VT-322'], None), color="black", label='FAULT 17')
plot3 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==0, \
                                                 final_dataset_filter['VT-322'], None), color="blue", label='NORMAL')

# ax1.set_xlabel('Timestamp')
ax2.set_ylabel('VT-322')
ax1.set_ylabel('Prediction (M5)') #color='red'
ax2.set_ylim([-2, 2])
ax1.yaxis.set_ticks(np.arange(0, 2, 1))

# Esquema para unificar na mesma caixa as legendas de ambos os eixos
lns = plot1+plot2+plot3
labs = [l.get_label() for l in lns]
ax2.legend(loc='upper right', prop={'size': 8})
# ax1.legend(lns, labs, loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=True, ncol=5)

fig.tight_layout()
# fig.savefig(saving_path + '/ingles_nb136_online_falha18.png', format='png', bbox_inches='tight');

### Falha 20

In [ ]:
fault = 20

In [ ]:
print(results_online_nb136_20_24[results_online_nb136_20_24['real'] == fault].time.max())
print(data_teste[data_teste['rotulos_multi'] == fault].time.max())
max_date = data_teste[data_teste['rotulos_multi'] == fault].time.max()

In [ ]:
fault_predict = results_online_nb136_20_24[results_online_nb136_20_24['time'] <= max_date]
fault_vars = data_teste[data_teste['time'] <= max_date][24:]

print(fault_predict.shape)
print(fault_vars.shape)

In [ ]:
final_dataset = fault_predict.merge(fault_vars, how='left', on=['time'])
final_dataset.shape

In [ ]:
final_dataset.isnull().sum().sum()

In [ ]:
final_dataset_filter = final_dataset#[30000:].copy()

In [ ]:
final_dataset_filter.rotulos_multi.unique()

In [ ]:
fig, ax1 = plt.subplots(figsize=(9,2), dpi=300)

ax2 = ax1.twinx()
plot1 = ax1.plot(final_dataset_filter['time'], final_dataset_filter['prediction'].astype('int'), color='red', label='PREDIÇÃO (M15)', linewidth=1)
plot2 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==fault, \
                                                 final_dataset_filter['VT-322'], None), color="black", label='FAULT 18')
plot3 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==0, \
                                                 final_dataset_filter['VT-322'], None), color="blue", label='NORMAL')

# ax1.set_xlabel('Timestamp')
ax2.set_ylabel('VT-322')
ax1.set_ylabel('Prediction (M5)') #color='red'
ax2.set_ylim([-4, 3])
ax1.yaxis.set_ticks(np.arange(0, 2, 1))

# Esquema para unificar na mesma caixa as legendas de ambos os eixos
lns = plot1+plot2+plot3
labs = [l.get_label() for l in lns]
ax2.legend(loc='upper left', prop={'size': 8})
# ax1.legend(lns, labs, loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=True, ncol=5)

fig.tight_layout()
# fig.savefig(saving_path + '/ingles_nb136_online_falha20.png', format='png', bbox_inches='tight');

### Falha 24

In [ ]:
fault = 24

In [ ]:
print(results_online_nb136_20_24[results_online_nb136_20_24['real'] == fault].time.max())
print(data_teste[data_teste['rotulos_multi'] == fault].time.max())
max_date = data_teste[data_teste['rotulos_multi'] == fault].time.max()

In [ ]:
fault_predict = results_online_nb136_20_24[results_online_nb136_20_24['time'] <= max_date]
fault_vars = data_teste[data_teste['time'] <= max_date][24:]

print(fault_predict.shape)
print(fault_vars.shape)

In [ ]:
final_dataset = fault_predict.merge(fault_vars, how='left', on=['time'])
final_dataset.shape

In [ ]:
final_dataset.isnull().sum().sum()

In [ ]:
final_dataset_filter = final_dataset[45900:105500].copy()
# final_dataset_filter['prediction'].iloc[:2070] = 0.0

In [ ]:
final_dataset_filter.rotulos_multi.unique()

In [ ]:
fig, ax1 = plt.subplots(figsize=(9,2.7), dpi=300)

ax2 = ax1.twinx()
plot1 = ax1.plot(final_dataset_filter['time'], final_dataset_filter['prediction'].astype('int'), color='red', label='PREDIÇÃO (M15)', linewidth=1)
plot2 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==fault, \
                                                 final_dataset_filter['VT-322'], None), color="black", label='FALHA 24')
plot3 = ax2.plot(final_dataset_filter['time'], np.where(final_dataset_filter["rotulos_multi"]==0, \
                                                 final_dataset_filter['VT-322'], None), color="blue", label='NORMAL')

ax1.set_xlabel('Timestamp')
ax2.set_ylabel('VT-322')
ax1.set_ylabel('Predição (M15)') #color='red'
ax2.set_ylim([-3, 4])
ax1.yaxis.set_ticks(np.arange(0, 2, 1))

# Esquema para unificar na mesma caixa as legendas de ambos os eixos
lns = plot1#+plot2+plot3
labs = [l.get_label() for l in lns]
ax2.legend(loc='upper center', prop={'size': 8})
ax1.legend(lns, labs, loc='upper center', bbox_to_anchor=(0.5, -0.3), fancybox=True, shadow=True, ncol=5, prop={'size': 8})

fig.tight_layout()
# fig.savefig(saving_path + '/nb136_online_falha24.png', format='png', bbox_inches='tight');

### Curvas de aprendizagem

In [ ]:
history_m15 = pd.read_csv(r"C:\Users\anaso\Desktop\workspace\doutorado\data\simple_model_NB136_6-CNN-history.csv", sep=';')
history_m15

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7,3), dpi=300)

history_m15_to_plot = history_m15.copy()

ax[0].plot(history_m15_to_plot['Unnamed: 0'], history_m15_to_plot['LOSS - TREINO'], '-go', label='Treino', markersize=2)
ax[0].plot(history_m15_to_plot['Unnamed: 0'], history_m15_to_plot['LOSS - VALID'], '-o', color='darkorange', label='Validação', markersize=2)
ax[0].legend(loc='upper right', prop={'size': 9})
ax[0].set(ylabel='Custo (loss)')
ax[0].set(xlabel='Épocas')

ax[1].plot(history_m15_to_plot['Unnamed: 0'], history_m15_to_plot['ACC - TREINO'], '-go', label='Treino', markersize=2)
ax[1].plot(history_m15_to_plot['Unnamed: 0'], history_m15_to_plot['ACC - VALID'], '-o', color='darkorange', label='Validação', markersize=2)
ax[1].legend(loc='lower right', prop={'size': 9})
ax[1].set(ylabel='Acurácia')
ax[1].set(xlabel='Épocas')

fig.tight_layout()
fig.savefig(saving_path + '/figura50_m15.png', format='png', bbox_inches='tight');